<a href="https://colab.research.google.com/github/westnowise/ApacheSpark_1/blob/main/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B06%EA%B8%B0_%EC%A1%B0%EC%84%9C%ED%98%84_DE%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset 다섯개 활용한 실습 과제
    - member.csv
    - point_his.csv
    - item_his.csv
    - study_his.csv
    - regdate_his.csv



### 환경 설정 및 세션

In [ ]:
!pip3 install pyspark

In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F

spark = SparkSession.builder \
        .master("local") \
        .appName("Colab") \
        .getOrCreate()

### 데이터 읽기

In [ ]:
member = spark.read.csv("/content/member.csv", header =True)
point = spark.read.csv("/content/point_his.csv", header =True)
item = spark.read.csv("/content/item_his.csv", header =True)
study = spark.read.csv("/content/study_his.csv", header =True)
regdate = spark.read.csv("/content/regdate_his.csv", header =True)

1. member 테이블에서 idx가 8000과 9000 사이인 row를 찾아주세요.(양 끝값 포함) (6)

In [ ]:
# member.distinct() 중복 없기 때문에 중복 제거 없이 진행
member.filter(F.col('idx').between(8000, 9000)).show()

+----+---+--------+-------+
| idx|sex|  status|  grade|
+----+---+--------+-------+
|8647| 남|유료회원|초4학년|
|8698| 남|유료회원|초2학년|
|8222| 남|유료회원|초4학년|
|8178| 남|유료회원|초1학년|
|8678| 남|유료회원|초4학년|
|8285| 여|유료회원|초4학년|
|8354| 남|유료회원|초3학년|
|8530| 여|유료회원|초3학년|
|8709| 남|유료회원|초1학년|
|8507| 남|유료회원|초2학년|
|8573| 남|유료회원|초3학년|
|8772| 남|유료회원|초3학년|
|8559| 남|유료회원|초3학년|
|8870| 여|유료회원|초5학년|
|8919| 여|유료회원|초4학년|
|8927| 여|유료회원|초3학년|
|8904| 여|유료회원|초5학년|
|8945| 여|유료회원|초6학년|
|8381| 여|유료회원|초4학년|
|8084| 여|유료회원|초6학년|
+----+---+--------+-------+
only showing top 20 rows



2. member 테이블에서 회원 상태별 인원수를 내림차순으로 보여주세요. (6)

In [ ]:
# member 테이블에서 distinct count 시 중복이 없기 때문에 중복 제거 없이 진행
member.groupby('status').count().orderBy(F.desc('count')).show()

+--------+-----+
|  status|count|
+--------+-----+
|유료회원|97980|
|학습만료|  383|
|  재구매|  357|
|    신규|  261|
|    이월|   57|
|    복회|   17|
|이관유료|   15|
|    취소|   10|
+--------+-----+



3. 학습만료 회원들이 공부한 과목 리스트를 TOP3를 뽑아주세요 (8)

In [ ]:
# 학습 만료 회원 필터링
member_status_end = member.filter(F.col('status') == '학습만료') # count() : 383

# 학습 만료 회원들이 공부한 과목 join 후 중복 제거
# 학습 만료 회원들의 과목에 중점을 둔 중복 제거 (시기가 다른 경우도 포함하여 제거)
join_study = member_status_end.join(study, ["idx"], "inner") # distinct() 시 count() : 365
join_study = join_study.dropDuplicates(subset = ['idx', 'pointnm']) # count() : 152

# 상위 3개 과목 선택
top3_subjects =join_study.groupBy(F.col('pointnm')).count() \
                    .orderBy(F.desc("count")) \
                    .limit(3)

# 결과 출력
top3_subjects.show()

+----------------+-----+
|         pointnm|count|
+----------------+-----+
|  AI 구구단 게임|   45|
|    AI서술형평가|   43|
|밀크T 지오그래픽|   21|
+----------------+-----+



4. 등록일(reg_date)이 2023.03.15일인 유료회원들이 가장 많이 착용한 아이템(codename) TOP3를 뽑아주세요 (8)

In [ ]:
# 등록일과 유료회원 필터링
regdate1 = regdate.filter(F.col('regdate') == '20230315') # 등록일 23.03.15, count() : 218
member1 = member.filter(F.col('status') == '유료회원') # status 유료회원, count() : 97980

# 필터링 된 데이터들을 item 테이블과 idx 기준으로 join
# 같은 날 같은 회원이 여러 번 같은 아이템 착용 가능하기 때문에 중복제거 없이 진행
join2 = item.join(regdate1, ["idx"], "inner") \
            .join(member1, ["idx"], "inner") # 348

# join -> idx, proc_ymd, codename이 동일한 데이터 중복 처리
join2 = join2.drop_duplicates(subset= ['idx', 'proc_ymd', 'codename']) # 245

top3_item = join2.groupBy('codename').count().orderBy(F.desc('count')).limit(3)
top3_item.show()

+----------+-----+
|  codename|count|
+----------+-----+
|      헤어|   47|
|      신발|   40|
|상태메시지|   38|
+----------+-----+



5. 레벨이 151~160 에 있는 유저들 중 딱 한 명씩만 등록한 날짜들을 구해주세요 (10)


In [ ]:
# 레벨 151 ~ 160 유저 item_lv 생성
item_lv = item.filter(F.col('lv').between(151, 160))

# regdate와 item_lv join
# idx, lv, regdate 중복 제거
date_lv = item_lv.join(regdate, ["idx"], "inner").drop_duplicates(['idx', 'lv', 'regdate'])

# 등록일 기준 유저 count
count_user_dates = date_lv.groupBy('regdate').agg(F.countDistinct('idx').alias('user_count')) # 113

# 등록한 유저가 딱 한 명인 날짜들 선택
one_user_dates = count_user_dates.filter(F.col('user_count') == 1)  # 19

# 결과 출력
one_user_dates.show()

+--------+----------+
| regdate|user_count|
+--------+----------+
|20230327|         1|
|20230505|         1|
|20230321|         1|
|20230129|         1|
|20230205|         1|
|20230428|         1|
|20230325|         1|
|20230330|         1|
|20230420|         1|
|20230320|         1|
|20230226|         1|
|20230411|         1|
|20230310|         1|
|20230316|         1|
|20230211|         1|
|20230410|         1|
|20230311|         1|
|20230331|         1|
|20230305|         1|
+--------+----------+



6. item_his 테이블에서 레벨이 null인 유저가 가장 많은 날짜를 구해주세요 (8)

In [ ]:
# 레벨이 null인 행들 필터링
# 'idx', 'proc_ymd' 중복 시 날짜 유저 구분 시 어렵기 때문에 제거
item_lv_Nan = item.filter(F.col('lv').isNull()).drop_duplicates(['idx', 'proc_ymd']) # 126

# 각 날짜별로 레벨이 null인 유저 수 계산
date_counts = item_lv_Nan.groupBy('proc_ymd').agg(F.countDistinct('idx').alias('user_count'))

# 가장 많은 유저가 있는 날짜 찾기
max_lv_nan_user_date = date_counts.orderBy(F.desc('user_count')).first()

# 결과 출력
max_lv_nan_user_date

Row(proc_ymd='20230505', user_count=7)

7. study_his 테이블의 pointnm 컬럼에 대해 공백을 모두 없애주세요 (8)

In [ ]:
# pointnm 에서 공백을 제거한 컬럼으로 대체한 새로운 테이블 생성
study_na_blank = study.withColumn('pointnm', F.regexp_replace('pointnm', '\s+', ''))

# 결과 확인
study_na_blank.show()

+-----+-------+--------+----------+
|  idx|proc_ym|proc_ymd|   pointnm|
+-----+-------+--------+----------+
|88311| 202306|20230628|한글스피치|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
| 8604| 202306|20230606| 중학3학년|
|89542| 202305|20230509| 중학3학년|
|23940| 202305|20230516| 중학1학년|
|23940| 202305|20230516| 중학1학년|
|40502| 202304|20230412| 중학1학년|
| 1741| 202304|20230405| 중학1학년|
|50161| 202304|20230428| 중학1학년|
+-----+-------+--------+----------+
only showing top 20 rows



8. member 테이블의 grade 컬럼에서 숫자만 뽑아 grade라는 컬럼을 재구성해주세요. (초3학년 -> 3) (8)
(UDF, slicing 사용 X)

In [ ]:
# 'grade' 컬럼에서 숫자 추출하여 'grade' 컬럼 재구성
member_grade = member.withColumn('grade_num', F.regexp_extract('grade', r'(\d+)', 1))

# 결과 확인
member_grade.show()

+-----+---+--------+-------+---------+
|  idx|sex|  status|  grade|grade_num|
+-----+---+--------+-------+---------+
| 6884| 여|유료회원|초3학년|        3|
| 6331| 남|유료회원|초3학년|        3|
|69294| 남|유료회원|초5학년|        5|
|31531| 여|유료회원|초1학년|        1|
|85784| 여|유료회원|초2학년|        2|
|58058| 여|유료회원|초3학년|        3|
|  777| 남|유료회원|초3학년|        3|
| 5482| 여|유료회원|초2학년|        2|
|63447| 남|유료회원|초2학년|        2|
|54957| 남|유료회원|초4학년|        4|
|55340| 남|유료회원|초2학년|        2|
|72887| 여|유료회원|초3학년|        3|
|40008| 여|유료회원|초3학년|        3|
|57551| 남|유료회원|초5학년|        5|
|58583| 여|유료회원|초3학년|        3|
|32219| 남|유료회원|초1학년|        1|
|13136| 여|유료회원|초4학년|        4|
|22122| 남|유료회원|초6학년|        6|
|18306| 남|유료회원|초4학년|        4|
|28057| 남|유료회원|초3학년|        3|
+-----+---+--------+-------+---------+
only showing top 20 rows



9. study_his 월별로 pointnm 컬럼에 대한 point 발생 수(count)를 오름차순으로 순번 매겨주세요 (8)

In [ ]:
# proc_ym에서 년도를 제외하여 proc_mm 컬럼 생성한 point_mon, study_mon
point_mon = point.withColumn('proc_mm',F.substring(F.col('proc_ym'), 5, 2))
study_mon = study.withColumn('proc_mm',F.substring(F.col('proc_ym'), 5, 2))

# 중복 제거를 위해 drop_duplicates 사용하여 중복 제거, 월별 point 발생 수에 초점을 맞춘 중복 제거
point_mm = point_mon.join(study_mon, ["idx", "proc_mm"], "inner") \
                    .drop_duplicates(['idx','proc_mm', 'point', 'pointnm'])

# point 발생 수 오름차순
point_mm = point_mm.groupby(F.col('proc_mm'), F.col('pointnm')) \
                   .agg(F.count("point").alias("point_count"))

# 순번 매기기
# window 변수
window_var = W.Window.partitionBy(F.col("pointnm")).orderBy(F.col("point_count").asc())
point_mm_rank = point_mm.withColumn('rank', F.rank().over(window_var))

# 결과 출력
point_mm_rank.show()

+-------+-----------------+-----------+----+
|proc_mm|          pointnm|point_count|rank|
+-------+-----------------+-----------+----+
|     05|   AI 구구단 게임|       2179|   1|
|     06|   AI 구구단 게임|       2275|   2|
|     04|   AI 구구단 게임|       2993|   3|
|     05|          AI 국어|        360|   1|
|     06|          AI 국어|        395|   2|
|     04|          AI 국어|        487|   3|
|     06|          AI 영어|        905|   1|
|     05|          AI 영어|       1265|   2|
|     04|          AI 영어|       1270|   3|
|     04|     AI서술형평가|        880|   1|
|     06|도전! AI 받아쓰기|        796|   1|
|     05|도전! AI 받아쓰기|        847|   2|
|     04|도전! AI 받아쓰기|       1120|   3|
|     05| 밀크T 지오그래픽|        436|   1|
|     06| 밀크T 지오그래픽|        520|   2|
|     04| 밀크T 지오그래픽|        717|   3|
|     04|         받아쓰기|          9|   1|
|     06|         받아쓰기|         10|   2|
|     06|        일기 쓰기|        645|   1|
|     05|        일기 쓰기|        866|   2|
+-------+-----------------+-----------+----+
only sh

10. pointnm 별로 획득한 point의 종류가 언더바(_)로 이어져서 보이도록 테이블을 만들어 주세요 (10)

In [ ]:
# idx를 활용하여 pointnm이 있는 study table과 point가 있는 point table join
join4 = study.join(point, ["idx"], "inner")

# pointnm 별로 획득한 point의 종류 _로 보이는 groupby 한 테이블 생성
point_underbar = join4.groupBy(F.col('pointnm')) \
                      .agg(F.concat_ws('_', F.collect_set('point')).alias('point_kind'))

# 결과 출력
point_underbar.show(truncate=False)

+-----------------+------------------------------------------------+
|pointnm          |point_kind                                      |
+-----------------+------------------------------------------------+
|일기 쓰기        |1000_4000_2000_5000_3500_4500_500_1500_3000_2500|
|학교 체험        |1000_4000_2000_5000_3500_4500_500_1500_3000_2500|
|중학 특강        |4000_1000_3500_5000_2000_4500_500_1500_3000_2500|
|AI 영어          |1000_4000_3500_2000_5000_4500_500_1500_3000_2500|
|도전! AI 받아쓰기|1000_4000_5000_2000_3500_4500_500_1500_3000_2500|
|학교 공부 맛보기 |1000_4000_3500_2000_5000_4500_500_1500_3000_2500|
|중학 신입생      |1000_4000_2000_3500_5000_4500_500_1500_3000_2500|
|AI 국어          |4000_1000_5000_3500_2000_4500_500_1500_3000_2500|
|밀크T 지오그래픽 |4000_1000_3500_5000_2000_4500_500_1500_3000_2500|
|중학 1학년       |1000_4000_3500_2000_5000_4500_500_1500_3000_2500|
|한글 스피치      |1000_4000_5000_3500_2000_4500_500_1500_3000_2500|
|받아쓰기         |1000_4000_2000_3500_5000_4500_500_1500_3000_2500|
|중학 3학년       |1000

11. point_his의 proc_ymd 컬럼의 날짜 표현 형식을 yyyy-mm-dd 형식이 되도록 바꿔주세요. (8)
(UDF, slicing 사용 X)

In [ ]:
# proc_ymd을 자리수 별로 끊어서 '-' 삽입
point_ymd_form = point.withColumn('proc_ymd', \
                        F.concat(F.substring(F.col('proc_ymd'), 1, 4), F.lit('-'), \
                                 F.substring(F.col('proc_ymd'), 5, 2), F.lit('-'), \
                                 F.substring(F.col('proc_ymd'), 7, 2)))

# 결과 출력
point_ymd_form.show()


+-----+-------+----------+-----+
|  idx|proc_ym|  proc_ymd|point|
+-----+-------+----------+-----+
|96465| 202306|2023-06-24| 1000|
|96465| 202306|2023-06-24|  500|
|87940| 202304|2023-04-05| 2000|
|87940| 202304|2023-04-05| 3500|
|87940| 202304|2023-04-05| 4000|
|87940| 202304|2023-04-05| 3000|
|87940| 202304|2023-04-05| 2500|
|88058| 202304|2023-04-05| 3000|
|88058| 202304|2023-04-05| 1000|
|88058| 202304|2023-04-05| 1500|
|88058| 202304|2023-04-05| 2500|
|88058| 202304|2023-04-05|  500|
|88058| 202304|2023-04-05| 2000|
|95844| 202306|2023-06-17| 1500|
|95844| 202306|2023-06-17| 1000|
|95050| 202306|2023-06-12| 3500|
|95050| 202306|2023-06-12| 4000|
|95050| 202306|2023-06-12| 4500|
|92560| 202305|2023-05-11| 1000|
|92560| 202305|2023-05-11|  500|
+-----+-------+----------+-----+
only showing top 20 rows



12. member의 status 컬럼에서 '회원'단어가 있으면 띄어쓰기하고 없으면 띄어쓰기 후 회원 단어 추가해주세요

In [ ]:
# member의 status 컬럼에서 '회원'단어가 있으면 띄어쓰기하고 없으면 띄어쓰기 후 회원 단어 추가해주세요
member.groupBy(F.col("status")).count().show()

# "회원" 단어를 포함하는 경우는 그대로 사용하고, 그렇지 않은 경우에는 " 회원"을 추가하여 새로운 컬럼 생성
member_inc = member.withColumn('status', F.when(F.col("status").like("%회원"), \
                                         F.concat(F.substring(F.col("status"), 1, 2), F.lit(" 회원"))) \
                                          .otherwise(F.concat(F.col("status"), F.lit(" 회원"))))

# 결과 출력
member_inc.groupBy(F.col("status")).count().show()

+--------+-----+
|  status|count|
+--------+-----+
|    신규|  261|
|유료회원|97980|
|학습만료|  383|
|이관유료|   15|
|  재구매|  357|
|    복회|   17|
|    취소|   10|
|    이월|   57|
+--------+-----+

+-------------+-----+
|       status|count|
+-------------+-----+
|    유료 회원|97980|
|    이월 회원|   57|
|    취소 회원|   10|
|    신규 회원|  261|
|  재구매 회원|  357|
|학습만료 회원|  383|
|이관유료 회원|   15|
|    복회 회원|   17|
+-------------+-----+

